In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import pickle
import re
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/drive"]

creds = ServiceAccountCredentials.from_json_keyfile_name("crudg.json", scope)
client = gspread.authorize(creds)
sheet = client.open("flightsched").get_worksheet(2)
dataframe = pd.DataFrame(sheet.get_all_records())


In [2]:

dataframe["Modeffdate"]= pd.to_datetime(dataframe["Modeffdate"], unit="s")
dataframe["moddisdate"]= pd.to_datetime(dataframe["moddisdate"], unit="s")
dataframe["moddeptime"]= pd.to_datetime(dataframe["moddeptime"], unit="s")
dataframe["mod_arr_time"]= pd.to_datetime(dataframe["mod_arr_time"], unit="s")
dataframe = dataframe.drop(columns=["Dept Time" ,"Arrv Time","Subfleet","Service Type","Eff Date","Dis Date","01-Jan-1970"])
dataframe["Freq"] = dataframe["Freq"].astype(str)
dataframe["Freq"] = dataframe["Freq"].str.replace(".","")
dataframe["engg_arr"]= False
dataframe["engg_dep"]=False
dataframe["deptype"]=False
dataframe["actual_days"]=""
cols = [ 'Dept Arp', 'engg_arr','engg_dep', 'deptype','Arvl Arp', 'Flt Desg',   'moddeptime', 'mod_arr_time'  ,'Text Comment', 'Freq','Block Time','Modeffdate', 'moddisdate',"actual_days"
,]
dataframe = dataframe[cols]

#  modyfying actualdays column for complete dataset 

for xray in range(0,len(dataframe)):
    bravo =  pd.to_datetime(str(dataframe.loc[xray,"Modeffdate"]))
    while dataframe.loc[xray,"moddisdate"] >= bravo:
        # one added as day of the week is different in pandas and data 
        if str(bravo.dayofweek + 1) in dataframe.loc[xray,"Freq"]:
            dataframe.loc[xray,"actual_days"]+= "/"+ str(bravo.day)
        bravo+= pd.DateOffset(1)

# adding columns for each day of month 
col_name_list=[]
nilList=[]
StrArr = "A"
StrDep ="D"
for india in range(1,32):
     col_name_list.append(StrDep + str(india))
     col_name_list.append(StrArr + str(india))
     nilList.append(False)
     nilList.append(False)
cols = cols +col_name_list
dataframe = pd.concat([dataframe,pd.DataFrame([nilList],index=dataframe.index,columns = col_name_list)], axis=1)



In [6]:

# first day of month set manually need to to check edge cases 
def remove_duplicate_flights (x_df):
      temp_dict ={}
      
      for xray in range(0,len(x_df)):
            if not (str(x_df.loc[xray,'Flt Desg']) in temp_dict):
                  temp_dict[x_df.loc[xray,'Flt Desg']] =xray
                  alpha = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"moddeptime"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)
                  beta = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"mod_arr_time"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)
                  for india in range(1,32):
                        if str(india) in x_df.loc[xray,'actual_days'].split("/"):
                              x_df.loc[xray,"D"+str(india)] = alpha
                              x_df.loc[xray,"A"+str(india)] = beta
                        alpha+=pd.DateOffset(1)
                        beta+=pd.DateOffset(1)

            elif (str(x_df.loc[xray,'Flt Desg']) in temp_dict):
                  alpha = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"moddeptime"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)
                  beta = pd.to_datetime(str(x_df.loc[xray,"Modeffdate"].date()) +" "+ x_df.loc[xray,"mod_arr_time"].strftime('%H:%M:%S'),format="%Y-%m-%d %H:%M:%S").replace(day=1)              
                  zulu = temp_dict[x_df.loc[xray,'Flt Desg']]
                  
                  for india in range(1,32):
                        if str(india) in x_df.loc[xray,'actual_days'].split("/"):
                              x_df.loc[zulu,"D"+str(india)] = alpha
                              x_df.loc[zulu,"A"+str(india)] = beta
                        alpha+=pd.DateOffset(1)
                        beta+=pd.DateOffset(1)

      
      
                 
remove_duplicate_flights(dataframe)
dataframe = dataframe.drop_duplicates(subset=['Flt Desg'])
dataframe = dataframe.reset_index(drop=True)

pickle.dump(dataframe, open("gsheet_pickle.p", "wb"))

alpha = pickle.load(open("gsheet_pickle.p", "rb"))

alpha.equals(dataframe)





True

In [5]:
# manual testing 
test_alpha = alpha.sample(n=5)

In [6]:
# manual testing continued as cell excution creates a new random sample 
onedf=dataframe.loc[dataframe["Text Comment"]=="A145"]
onedf.loc[:,["Flt Desg","A11","A12","A13","D17","D19","D21","D18","D20"]]

,Flt Desg,A11,A12,A13,D17,D19,D21,D18,D20
210,6E 5998,2021-02-11 13:00:00,2021-02-12 13:00:00,2021-02-13 13:00:00,2021-02-17 10:25:00,2021-02-19 10:25:00,2021-02-21 10:25:00,2021-02-18 10:25:00,2021-02-20 10:25:00
211,6E 5997,2021-02-11 16:25:00,2021-02-12 16:25:00,2021-02-13 16:25:00,2021-02-17 13:50:00,2021-02-19 13:50:00,2021-02-21 13:50:00,2021-02-18 13:50:00,2021-02-20 13:50:00
212,6E 0224,2021-02-11 18:20:00,2021-02-12 18:20:00,2021-02-13 18:20:00,2021-02-17 17:40:00,2021-02-19 17:40:00,2021-02-21 17:40:00,2021-02-18 17:10:00,2021-02-20 17:10:00
213,6E 0354,2021-02-11 21:25:00,2021-02-12 21:25:00,2021-02-13 21:25:00,2021-02-17 21:00:00,2021-02-19 21:00:00,2021-02-21 21:00:00,2021-02-18 20:25:00,2021-02-20 20:25:00


In [7]:



onedf=dataframe.loc[dataframe["Text Comment"]=="A10"]

# sorting and operting on one rotation 

onedf =onedf.sort_values("moddeptime")


onedf["deptype"].iloc[0] ="baseDep"
onedf["deptype"].iloc[-1] ="baseArr"
mylist = onedf['Flt Desg'].values.tolist()
indices_base_dep = [i for i, x in enumerate(mylist) if x == mylist[0]]
indices_base_arr = [i for i, x in enumerate(mylist) if x == mylist[-1]]
for india in indices_base_dep:
    onedf["deptype"].iloc[india] ="baseDep"
for xray in indices_base_arr:
    onedf["deptype"].iloc[xray] ="baseArr"


# adding  a duplicate row at the bottom for base arival Aircraft 
is_Base_Arrival = onedf["deptype"] =="baseArr"
onedf_try=onedf[is_Base_Arrival]
onedf=onedf.append(onedf_try,ignore_index=True)


# modifying the eng_arr and eng_dep for one rotation 
for xray in range(0,len(onedf)):
    if onedf.loc[xray,"deptype"] == "baseDep":
        onedf.loc[xray,"engg_arr"] = "baseAcft"
        onedf.loc[xray,"engg_dep"] = onedf.loc[xray,"moddeptime"]
    elif onedf.loc[xray,"deptype"] != "baseDep" and onedf.loc[xray,"deptype"] != "baseArr":
        onedf.loc[xray,"engg_arr"] = onedf.loc[xray-1,"mod_arr_time"] 
        onedf.loc[xray,"engg_dep"] = onedf.loc[xray,"moddeptime"] 

